In [1]:
import pandas as pd
import os
import numpy as np
import pandas as pd
import joblib
from functools import reduce
from imblearn.over_sampling import RandomOverSampler

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, StratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.multiclass import OneVsRestClassifier
#from skmultilearn.model_selection import MultiLabelStratifiedKFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold


In [2]:
feat_breed = pd.read_pickle('../features/breed.pkl')
feat_breed_v2 = pd.read_pickle('../features/breed_v2.pkl')

feat_breed_top50 = pd.read_pickle('../features/breed_top50.pkl')
feat_breed_top50_v2 = pd.read_pickle('../features/breed_top50_v2.pkl')

feat_breed_group = pd.read_pickle('../features/breed_group.pkl')
feat_sub_breed = pd.read_pickle('../features/sub_breed.pkl')
feat_breed_type = pd.read_pickle('../features/breed_type.pkl')

feat_breed_pure_or_mix = pd.read_pickle('../features/breed_pure_or_mix.pkl')

feat_age = pd.read_csv('../features/age_with_id.csv')
feat_sex = pd.read_csv('../features/one_hot_encoded_sex_with_id.csv')

# Climate
feat_HotWheater = pd.read_csv('../features/HotMonths_with_dog_id.csv')
feat_residence = pd.read_csv('../features/Residence_with_dog_id.csv')

# Physical Activity
feat_pa_total_hours = pd.read_csv('../features/PhysicalActivity_total_hours.csv')
feat_pa_surface = pd.read_csv('../features/PhysicalActivity_surface.csv')

# Owner Demographics
feat_od_income = pd.read_csv('../features/od_income.csv')

feat_disease_input = pd.read_csv('../features/one_hot_encoded_disease_input.csv')
feat_disease_output_binary = pd.read_csv('../features/disease_output_binary.csv')
feat_disease_output = pd.read_csv('../features/disease_output.csv')

#Diet
feat_diet = pd.read_pickle('../features/diet_merged.pkl')

# Output
feat_disease_output_binary = pd.read_csv('../features/disease_output_binary.csv')
feat_disease_output = pd.read_csv('../features/disease_output.csv')

features_list = [
    #feat_breed,
    #feat_breed_v2,
    #feat_breed_top50,
    feat_breed_top50_v2,
    feat_breed_group,
    feat_sub_breed,
    feat_breed_type,
    feat_breed_pure_or_mix,
    feat_age,
    feat_sex,
    feat_HotWheater,
    feat_residence,
    feat_pa_total_hours,
    feat_pa_surface,
    feat_od_income,
    feat_diet,
    #feat_disease_input,
    #feat_disease_output_binary,
    #feat_disease_output
]

In [3]:
exp1 = [feat_breed_group]
exp2 = [feat_breed_top50_v2]
exp3 = [feat_age]
exp4 = [feat_sex,feat_breed_type]
exp5 = [feat_sub_breed]
exp6 = [feat_breed_pure_or_mix]
exp7 = [feat_breed_top50_v2,feat_breed_group]
exp8 = [feat_breed_top50_v2,feat_breed_group,feat_age,feat_sex]
exp9 = [feat_breed_top50_v2,feat_breed_group,feat_age,feat_sex,feat_pa_surface,feat_pa_total_hours]
exp10 = [feat_breed_top50_v2,feat_breed_group,feat_age,feat_sex,feat_od_income,feat_HotWheater,feat_residence]
exp11 = [feat_breed_top50_v2,feat_breed_group,feat_age,feat_sex,feat_od_income]
exp12 = [feat_breed_top50_v2,feat_breed_group,feat_age,feat_sex,feat_od_income,feat_HotWheater,feat_residence,feat_pa_surface,feat_pa_total_hours]
exp13 = [feat_breed_top50_v2,feat_breed_group,feat_HotWheater,feat_residence,feat_pa_surface,feat_pa_total_hours]
exp14 = [feat_breed_top50_v2,feat_breed_group,feat_HotWheater,feat_residence,feat_pa_surface,feat_pa_total_hours,feat_age,feat_sex,feat_diet]
exp15 = [feat_breed_top50_v2,feat_breed_group,feat_HotWheater,feat_residence,feat_pa_surface,feat_pa_total_hours,feat_age,feat_sex,feat_diet]
exp16 = [feat_breed_top50_v2,feat_breed_group,feat_HotWheater,feat_residence,feat_pa_surface,feat_pa_total_hours,feat_age,feat_sex,feat_breed_pure_or_mix,feat_breed_type,feat_sub_breed,feat_diet]

In [4]:
experiments = {
    'exp1': exp1,
    'exp2': exp2,
    'exp3': exp3,
    'exp4': exp4,
    'exp5': exp5,
    'exp6': exp6,
    'exp7': exp7,
    'exp8': exp8,
    'exp9': exp9,
    'exp10': exp10,
    'exp11': exp11,
    'exp12': exp12,
    'exp13': exp13,
    'exp15': exp15,
    'exp16': exp16
}
results = {}

In [21]:
X = data.drop(['dog_id'] + ['hs_health_conditions_' + condition for condition in [
    'eye', 'ear', 'oral', 'skin', 'cardiac', 'respiratory', 'gastrointestinal',
    'liver', 'kidney', 'reproductive', 'orthopedic', 'neurological', 'endocrine',
    'hematologic', 'immune', 'infectious_disease', 'toxin_consumption', 'trauma', 'cancer'
]], axis=1)
y_columns = ['hs_health_conditions_' + condition for condition in [
    'eye', 'ear', 'oral', 'skin', 'cardiac', 'respiratory', 'gastrointestinal',
    'liver', 'kidney', 'reproductive', 'orthopedic', 'neurological', 'endocrine',
    'hematologic', 'immune', 'infectious_disease', 'toxin_consumption', 'trauma', 'cancer'
]]
y = data[y_columns]

# Convert y to a binary format
y_binary = (y == 1)

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Initialize model MLP
mlp_model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, alpha=0.001, solver='adam', random_state=42)

# Wrapping MLP model using OneVsRestClassifier
ovr_classifier = OneVsRestClassifier(mlp_model)

# Initialize MultilabelStratifiedKFold
n_splits = 5  # 可以根据需要调整分割数
ml_stratified_kfold = MultilabelStratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Dictionary for storing AUC scores and sample counts
auc_scores_per_condition = {condition: [] for condition in y_columns}
sample_counts_per_condition = {condition: [] for condition in y_columns}

# fold iteration
for fold, (train_index, val_index) in enumerate(ml_stratified_kfold.split(X_scaled, y_binary)):
    X_train, X_val = X_scaled[train_index], X_scaled[val_index]
    y_train, y_val = y_binary.iloc[train_index], y_binary.iloc[val_index]

    # train model
    ovr_classifier.fit(X_train, y_train)

    # predict on validation-set
    y_pred_proba = ovr_classifier.predict_proba(X_val)

    # Calculate AUC scores for each disease
    auc_scores = [roc_auc_score(y_val[column], y_pred_proba[:, i]) for i, column in enumerate(y_val.columns)]

    # print and save AUC scores for each condition
    print(f"\nFold {fold+1} AUC Scores for Diseases:")
    for i, auc_score in enumerate(auc_scores):
        condition = y_val.columns[i]
        print(f"{condition}: {auc_score}")
        auc_scores_per_condition[condition].append(auc_score)
        sample_count = y_val[condition].sum()  # 计算该状况的阳性样本计数
        sample_counts_per_condition[condition].append(sample_count)


Fold 1 AUC Scores for Diseases:
hs_health_conditions_eye: 0.6711226444651028
hs_health_conditions_ear: 0.8027864090146773
hs_health_conditions_oral: 0.9648775247735664
hs_health_conditions_skin: 0.7629029802119873
hs_health_conditions_cardiac: 0.8739820043171836
hs_health_conditions_respiratory: 0.5467799315758021
hs_health_conditions_gastrointestinal: 0.5194915134709197
hs_health_conditions_liver: 0.5527234233438229
hs_health_conditions_kidney: 0.576539711985615
hs_health_conditions_reproductive: 0.5837613142469795
hs_health_conditions_orthopedic: 0.7577472419736013
hs_health_conditions_neurological: 0.5268245739359143
hs_health_conditions_endocrine: 0.5331919575416574
hs_health_conditions_hematologic: 0.4990403149735471
hs_health_conditions_immune: 0.5759053591854444
hs_health_conditions_infectious_disease: 0.7316788177743592
hs_health_conditions_toxin_consumption: 0.7119209299769564
hs_health_conditions_trauma: 0.8870009876907
hs_health_conditions_cancer: 0.5434083638288568

Fold 2

In [22]:
# Calculate the mean AUC score for each condition
average_auc_scores_per_condition = {condition: sum(scores)/len(scores) for condition, scores in auc_scores_per_condition.items()}

# Print the average AUC score for each condition
print("\nAverage AUC Scores for Each Health Condition:")
for condition, average_auc in average_auc_scores_per_condition.items():
    print(f"{condition}: {average_auc:.4f}")  # 使用四位小数精度



Average AUC Scores for Each Health Condition:
hs_health_conditions_eye: 0.6724
hs_health_conditions_ear: 0.7987
hs_health_conditions_oral: 0.9646
hs_health_conditions_skin: 0.7619
hs_health_conditions_cardiac: 0.8751
hs_health_conditions_respiratory: 0.5571
hs_health_conditions_gastrointestinal: 0.5114
hs_health_conditions_liver: 0.5694
hs_health_conditions_kidney: 0.5587
hs_health_conditions_reproductive: 0.5806
hs_health_conditions_orthopedic: 0.7525
hs_health_conditions_neurological: 0.5377
hs_health_conditions_endocrine: 0.5523
hs_health_conditions_hematologic: 0.5273
hs_health_conditions_immune: 0.5288
hs_health_conditions_infectious_disease: 0.7280
hs_health_conditions_toxin_consumption: 0.7116
hs_health_conditions_trauma: 0.8861
hs_health_conditions_cancer: 0.5612


In [6]:
results = {}

for exp_name, exp_features in experiments.items():
    # Merge DataFrames iteratively using reduce
    input_features = reduce(lambda left, right: pd.merge(left, right, on='dog_id'), exp_features)

    # Merge with disease output feature
    data = pd.merge(input_features, feat_disease_output_binary, on='dog_id')

    # Handle missing values and separate features and labels
    rows_with_nan = data[data.isna().any(axis=1)]
    columns_with_nan = data.columns[data.isna().any()].tolist()
    rows_with_nan[columns_with_nan]
    
    X = data.drop(['dog_id'] + ['hs_health_conditions_' + condition for condition in [
        'eye', 'ear', 'oral', 'skin', 'cardiac', 'respiratory', 'gastrointestinal',
        'liver', 'kidney', 'reproductive', 'orthopedic', 'neurological', 'endocrine',
        'hematologic', 'immune', 'infectious_disease', 'toxin_consumption', 'trauma', 'cancer'
    ]], axis=1)

    y_columns = ['hs_health_conditions_' + condition for condition in [
        'eye', 'ear', 'oral', 'skin', 'cardiac', 'respiratory', 'gastrointestinal',
        'liver', 'kidney', 'reproductive', 'orthopedic', 'neurological', 'endocrine',
        'hematologic', 'immune', 'infectious_disease', 'toxin_consumption', 'trauma', 'cancer'
    ]]
    y = data[y_columns]

    y_binary = (y == 1)

    # normalize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    num_features = X.shape[1]
    if num_features <= 50:
        hidden_layer_sizes = (50,)
    elif num_features <= 100:
        hidden_layer_sizes = (100, 50)
    else:
        hidden_layer_sizes = (100, 100, 50)

    # Initialize the MLP model
    mlp_model = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, max_iter=1000, random_state=42)
    # Wrap the model with OneVsRestClassifier
    ovr_classifier = OneVsRestClassifier(mlp_model)

    # Initialize MultilabelStratifiedKFold
    n_splits = 5
    ml_stratified_kfold = MultilabelStratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    # auc_scores_per_fold = []

    # Initialize the dictionary that stores the results
    dataset_results = {'train': [], 'validation': []}

    # auc_scores_per_condition = {condition: [] for condition in y_columns}
    # sample_counts_per_condition = {condition: [] for condition in y_columns}

    # Iterate through the splits
    for fold, (train_index, val_index) in enumerate(ml_stratified_kfold.split(X_scaled, y_binary)):
        X_train, X_val = X_scaled[train_index], X_scaled[val_index]
        y_train, y_val = y_binary.iloc[train_index], y_binary.iloc[val_index]

        # Train the model
        ovr_classifier.fit(X_train, y_train)

        # Make predictions on the validation and train set
        y_train_pred_proba = ovr_classifier.predict_proba(X_train)
        y_val_pred_proba = ovr_classifier.predict_proba(X_val)

        # Calculate the AUC score for each disease
        train_auc_scores = [roc_auc_score(y_train.iloc[:, i], y_train_pred_proba[:, i]) for i in range(y_train.shape[1])]
        val_auc_scores = [roc_auc_score(y_val[column], y_val_pred_proba[:, i]) for i, column in enumerate(y_val.columns)]
        # print(f"\nFold {fold+1} AUC Scores for Diseases:")
        # for i, auc_score in enumerate(auc_scores):
        #     condition = y_val.columns[i]
        #     print(f"{condition}: {auc_score}")
            # auc_scores_per_condition[condition].append(auc_score)
            # sample_count = y_val[condition].sum()  # Calculate the count of positive samples for the condition
            # sample_counts_per_condition[condition].append(sample_count)

        dataset_results['train'].append(np.mean(train_auc_scores))
        dataset_results['validation'].append(np.mean(val_auc_scores))
        # auc_scores_per_fold.append(np.mean(auc_scores))

    # Calculate the average AUC score for the entire experiment
    # average_auc_score = np.mean(auc_scores_per_fold)
    # results[exp_name] = average_auc_score
    average_auc_train = np.mean(dataset_results['train'])
    average_auc_validation = np.mean(dataset_results['validation'])
    results[exp_name] = {'train': average_auc_train, 'validation': average_auc_validation}

    # print results for train and validation
    print(f"{exp_name}:")
    print(f"    Average AUC on Train = {average_auc_train:.4f}")
    print(f"    Average AUC on Validation = {average_auc_validation:.4f}")

# Print the results for different experiments
for exp_name, auc_score in results.items():
    print(f"{exp_name}: Average AUC = {auc_score:.4f}")

exp1:
    Average AUC on Train = 0.5776
    Average AUC on Validation = 0.5609
exp2:
    Average AUC on Train = 0.6641
    Average AUC on Validation = 0.5729
exp3:
    Average AUC on Train = 0.6735
    Average AUC on Validation = 0.6728
exp4:
    Average AUC on Train = 0.5784
    Average AUC on Validation = 0.5570
exp5:
    Average AUC on Train = 0.5863
    Average AUC on Validation = 0.5655
exp6:
    Average AUC on Train = 0.5237
    Average AUC on Validation = 0.5224
exp7:
    Average AUC on Train = 0.7023
    Average AUC on Validation = 0.5699
exp8:
    Average AUC on Train = 0.8727
    Average AUC on Validation = 0.6374
exp9:
    Average AUC on Train = 0.9888
    Average AUC on Validation = 0.6007
exp10:
    Average AUC on Train = 0.9829
    Average AUC on Validation = 0.5959
exp11:
    Average AUC on Train = 0.9378
    Average AUC on Validation = 0.6109
exp12:
    Average AUC on Train = 0.9991
    Average AUC on Validation = 0.5981
exp13:
    Average AUC on Train = 0.9841
    Aver

TypeError: unsupported format string passed to dict.__format__